# ETL Pipeline for Bison Data

This notebook cleans lab results data for various blood tests performed on wild bison in the State of Utah. Raw data is in .xlsx format with individual sheets for each test, having been extracted with Tabula from PDF results provided by the lab. Future improvements to this pipeline could include extracting directly from the PDF using python libraries. 

Clean data is organized into a single table, exported to .xlsx for use by the Utah Division of Wildlife Resources, and loaded into a SQLite database for future data analysis. 

In [ ]:
#import libraries
